In [1]:
import os
from timeit import default_timer as timer

import numpy as np
from scipy.stats import unitary_group
from opt_einsum import contract

from QDDPM_tf import MultiQubitDiffusionModel, QDDPM
from QDDPM_tf import naturalDistance, WassDistance

import tensorflow as tf
import tensorflow.math as tfm

2023-09-29 02:43:13.614755: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-29 02:43:13.644199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 02:43:14.066646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-29 02:43:14.566001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-29 02:43:14.600

# Correlated noise training

In [2]:
def noiseTraining_t(model, t, inputs_T, params_tot, Ndata, epochs, dis_measure='nat', dis_params={}):
    '''
    backward training of QDDPM for correlated noise
    Args:
    model: the QDDPM
    t: diffusion step
    params_tot: collection of PQC parameters for steps > t 
    Ndata: number of samples in training data set
    epochs: number of iterations
    dis_measure: the distance measure to compare two distributions of quantum states
    dis_params: potential hyper-parameters for distance measure
    '''
    
    input_tplus1 = model.prepareInput_t(inputs_T, params_tot, t, Ndata) # prepare input
    states_diff = model.states_diff
    loss_hist = [] # record of training history
    f_hist = [] # record of std of bloch-y cooredinates

    tf.random.set_seed(None)
    params_t = tf.Variable(tf.random.normal([2 * model.n_tot * model.L]))
    # set optimizer and learning rate decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        0.01, 200, 0.8, staircase=True)
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

    for _ in range(epochs):
        indices = np.random.choice(states_diff.shape[1], size=Ndata, replace=False)
        true_data = states_diff[t, indices]
        
        with tf.GradientTape() as tape:
            output_t = model.backwardOutput_t(input_tplus1, params_t)
            if dis_measure == 'nat':
                # natural distance
                loss = naturalDistance(output_t, true_data)
            elif dis_measure == 'wd':
                # Wassastein distance
                loss = WassDistance(output_t, true_data)

        grads = tape.gradient(loss, [params_t])
        optimizer.apply_gradients(zip(grads, [params_t]))

        loss_hist.append(tf.stop_gradient(loss)) # record the current loss
        f_hist.append(tf.math.reduce_mean(tf.abs(tf.stop_gradient(output_t)[:,2])**2))

    return tf.stop_gradient(params_t), tf.squeeze(tf.stack(loss_hist)), tf.squeeze(tf.stack(f_hist))

In [ ]:
# training
n, na = 2, 2
T = 20
L = 6
Ndata = 500
epochs = 2500
repeat = 5
method = 'nat'

diffModel = MultiQubitDiffusionModel(n, T, Ndata)
inputs_T = diffModel.HaarSampleGeneration(Ndata, seed=22)

model = QDDPM(n=n, na=na, T=T, L=L)
states_diff = np.load('data/corrNoiseDiff_n%dT%d_N5000.npy'%(n, T))
model.set_diffusionSet(states_diff)

data_path = "noise/record_%s/" %method
if not os.path.exists(data_path):
    os.makedirs(data_path)

for t in range(T-1, -1, -1):
    params_tot = np.zeros((20, 2*(n+na)*L))
    for tt in range(t+1, 20):
        params_tot[tt] = np.load('noise/record_%s/QDDPMcorrNoiseparams_n%dna%dT%dL%d_t%d_%s.npy'
                                % (method, n, na, T, L, tt, method))

    params_all = np.zeros((repeat, 2*(n+na)**L))
    loss_all = np.zeros((repeat, epochs))
    f_all = np.zeros((repeat, epochs))
    for trial in range(repeat):
        params, loss, f = noiseTraining_t(
            model, t, inputs_T, params_tot, Ndata, epochs, method)
        params_all[trial] = params.numpy()
        loss_all[trial] = loss.numpy()
        f_all[trial] = f.numpy()
        print(t, trial, loss_all[trial, -1])

    idx = np.argmin(loss_all[:, -1])
    np.save('noise/record_%s/QDDPMcorrNoiseparams_n%dna%dT%dL%d_t%d_%s.npy'
            % (method, n, na, T, L, t, method), params_all[idx])
    np.save('noise/record_%s/QDDPMcorrNoiseloss_n%dna%dT%dL%d_t%d_%s.npy'
            % (method, n, na, T, L, t, method), loss_all[idx])
    np.save('noise/record_%s/QDDPMcorrNoisef_n%dna%dT%dL%d_t%d_%s.npy'
            % (method, n, na, T, L, t, method), f_all[idx])

    print('corr-noise, na=%d, t=%d, min loss=%s, F10=%s'
          % (na, t, loss_all[idx, -1], f_all[idx, -1]))


# Identical product of circle training

In [2]:
def idProdTraining_t(model, t, inputs_T, params_tot, Ndata, epochs, dis_measure='wd', dis_params={}):
    '''
    backward training of QDDPM at step t for identical product state
    Args:
    model: the QDDPM
    t: diffusion step
    params_tot: collection of PQC parameters for steps > t 
    Ndata: number of samples in training data set
    epochs: number of iterations
    dis_measure: the distance measure to compare two distributions of quantum states
    dis_params: potential hyper-parameters for distance measure
    '''
    
    input_tplus1 = model.prepareInput_t(inputs_T, params_tot, t, Ndata) # prepare input
    states_diff = model.states_diff
    loss_hist = [] # record of training history
    y1_hist = [] # record of bloch-y coordinates for first qubit
    y2_hist = [] # record of bloch-y coordinates for second qubit
    # Pauli-y operator on first/second qubit
    sy = np.array([[0,-1j], [1j, 0]])
    sy1 = tf.cast(tf.convert_to_tensor(np.kron(sy, np.eye(2))), dtype = tf.complex64)
    sy2 = tf.cast(tf.convert_to_tensor(np.kron(np.eye(2), sy)), dtype = tf.complex64)

    if dis_measure == 'nat':
        # natural distance
        loss_func = naturalDistance
    elif dis_measure == 'wd':
        # Wassastein distance
        loss_func = WassDistance

    tf.random.set_seed(None)
    params_t = tf.Variable(tf.random.normal([2 * model.n_tot * model.L]))
    # set optimizer and learning rate decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        0.01, 200, 0.8, staircase=True)
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

    for _ in range(epochs):
        indices = np.random.choice(states_diff.shape[1], size=Ndata, replace=False)
        true_data = states_diff[t, indices]
        
        with tf.GradientTape() as tape:
            output_t = model.backwardOutput_t(input_tplus1, params_t)
            loss = loss_func(output_t, true_data)

        grads = tape.gradient(loss, [params_t])
        optimizer.apply_gradients(zip(grads, [params_t]))

        loss_hist.append(tf.stop_gradient(loss)) # record the current loss
        y1 = tfm.real(contract('mi, ij, mj-> m',  tfm.conj(tf.stop_gradient(output_t)), sy1, tf.stop_gradient(output_t)))
        y2 = tfm.real(contract('mi, ij, mj-> m',  tfm.conj(tf.stop_gradient(output_t)), sy2, tf.stop_gradient(output_t)))
        y1_hist.append(y1)
        y2_hist.append(y2)

    y1_hist = tf.stack(y1_hist)
    y2_hist = tf.stack(y2_hist)
    
    return tf.stop_gradient(params_t), tf.squeeze(tf.stack(loss_hist)), y1_hist, y2_hist

In [3]:
def prodHaarStates(n, Ndata, seed):
    '''
    generate two-qubit product states of one-qubit Haar random states
    '''
    np.random.seed(seed)
    state_qs = unitary_group.rvs(2, size=n*Ndata)[:, :, 0]
    state_qs = np.split(state_qs, n, axis=0)
    states = state_qs[0]
    for i in range(1, n):
        states = contract('mi, mj->mij', states, state_qs[i]).reshape((Ndata, 2**(i + 1)))
        
    return states

In [5]:
n, na = 2, 4
T = 20
L = 10
Ndata = 500
epochs = 2500
repeat = 5
method = 'wd'

inputs_T = prodHaarStates(n, Ndata, seed=22)

model = QDDPM(n=n, na=na, T=T, L=L)
states_diff = np.load('data/idProdDiff_n%dT%d_N5000.npy'%(n, T))
model.set_diffusionSet(states_diff)

data_path = "product/record_%s/" %method
if not os.path.exists(data_path):
    os.makedirs(data_path)

for t in range(T-1, -1, -1):
    params_tot = np.zeros((20, 2*(n+na)*L))
    for tt in range(t+1, 20):
        params_tot[tt] = np.load('product/record_%s/QDDPMidProdparams_n%dna%dT%dL%d_t%d_%s.npy'
                                %(method, n, na, T, L, tt, method))

    params_all = np.zeros((repeat, 2*(n + na)*L))
    loss_all = np.zeros((repeat, epochs))
    y1_all = np.zeros((repeat, epochs, Ndata))
    y2_all = np.zeros((repeat, epochs, Ndata))

    start_time = timer()
    for trial in range(repeat):
        params, loss, y1, y2 = idProdTraining_t(model, t, inputs_T, params_tot, Ndata, epochs, method)
        params_all[trial] = params.numpy()
        loss_all[trial] = loss.numpy()
        y1_all[trial] = y1.numpy()
        y2_all[trial] = y2.numpy()
        curr_time = timer()
        print(t, trial, loss_all[trial, -1], curr_time - start_time)
        
    idx = np.argmin(loss_all[:, -1])
    np.save('product/record_%s/QDDPMidProdparams_n%dna%dT%dL%d_t%d_%s.npy'
            % (method, n, na, T, L, t, method), params_all[idx])
    np.save('product/record_%s/QDDPMidProdloss_n%dna%dT%dL%d_t%d_%s.npy'
            % (method, n, na, T, L, t, method), loss_all[idx])
    np.save('product/record_%s/QDDPMidPrody1_n%dna%dT%dL%d_t%d_%s.npy'
            % (method, n, na, T, L, t, method), y1_all[idx])
    np.save('product/record_%s/QDDPMidPrody2_n%dna%dT%dL%d_t%d_%s.npy'
            % (method, n, na, T, L, t, method), y2_all[idx])

    print('id-product, na=%d, t=%d, min loss=%s, (y1-y2)^2=%s'
          % (na, t, loss_all[idx, -1], np.mean((y1_all[idx, -1]-y2_all[idx, -1])**2)))

    

19 0 0.0013423562049865723 173.09123904599983
19 1 0.00344240665435791 336.44428790800157


KeyboardInterrupt: 

# Area-law entanglement training